In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import iris
import tobac
import sys
from glob import glob
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as ani
from IPython.display import HTML
from cartopy import crs as ccrs
from cartopy import util as cutil
from cartopy.mpl.ticker import LatitudeFormatter, LongitudeFormatter

%matplotlib inline

In [ ]:
mode = "Moisture Mode"

diri = "/scratch/gb02/mr4682/data/regridded/UM/not_removing_tc/"

if mode == "Moisture Mode":
    mode_str = "moisture.mode"
    mode_var_name = "moisture_mode"
    v_max = 9.0
elif mode == "Mixed System":
    mode_str = "mixed.system"
    mode_var_name = "mixed_system"
    v_max = 27.0
elif mode == "IG Wave":
    mode_str = "ig.wave"
    mode_var_name = "ig_wave"
    v_max = 50.0
elif mode == "Tropical Cyclone":
    modes_str = "tc"
    mode_var_name = mode_str
    v_max = 10.0
else:
    sys.exit(f"There is no mode named {mode}!")

filei = f"olr.zoom10.to.0p25deg.{mode_str}.nc"

In [2]:
diri = "/scratch/gb02/mr4682/data/regridded/UM/not_removing_tc/"

ds1 = xr.open_dataset(f"{diri}olr.zoom10.to.0p25deg.moisture.mode.nc")
ds2 = xr.open_dataset(f"{diri}olr.zoom10.to.0p25deg.mixed.system.nc")
ds3 = xr.open_dataset(f"{diri}olr.zoom10.to.0p25deg.ig.wave.nc")

In [3]:
time_start = "2020-12-01 00:00:00"
time_end = "2021-02-28 21:00:00"

In [4]:
time = ds1["time"].sel(time=slice(time_start, time_end))
latitude = ds1["latitude"]
longitude = ds1["longitude"]

In [5]:
olr1 = ds1[f"rlut_moisture_mode"].sel(time=slice(time_start, time_end)).compute()
olr2 = ds2[f"rlut_mixed_system"].sel(time=slice(time_start, time_end)).compute()
olr3 = ds3[f"rlut_ig_wave"].sel(time=slice(time_start, time_end)).compute()

In [6]:
olr1_stddev = olr1.std(ddof=1)
olr2_stddev = olr2.std(ddof=1)
olr3_stddev = olr3.std(ddof=1)

In [7]:
olr1_standardised = olr1 / olr1_stddev
olr2_standardised = olr2 / olr2_stddev
olr3_standardised = olr3 / olr3_stddev

olr1_standardised = olr1_standardised.fillna(0.0)
olr2_standardised = olr2_standardised.fillna(0.0)
olr3_standardised = olr3_standardised.fillna(0.0)

In [8]:
dxy = np.radians(longitude.values[1] - longitude.values[0]) * 6.371e+06
dt = (time.values[1] - time.values[0]) / np.timedelta64(1, "s")

In [9]:
parameters_features = {}
parameters_features["threshold"] = [-2.0, -2.5, -3.0, -3.5, -4.0]
parameters_features["target"] = "minimum"
parameters_features["position_threshold"] = "weighted_diff"
parameters_features["sigma_threshold"] = 1.0
parameters_features["n_min_threshold"] = 4
parameters_features["PBC_flag"] = "hdim_2"

In [11]:
Features1 = tobac.feature_detection_multithreshold(olr1_standardised, dxy, **parameters_features)
Features2 = tobac.feature_detection_multithreshold(olr2_standardised, dxy, **parameters_features)
Features3 = tobac.feature_detection_multithreshold(olr3_standardised, dxy, **parameters_features)

In [12]:
parameters_linking = {}
parameters_linking["stubs"] = 3
parameters_linking["method_linking"] = "predict"
parameters_linking["PBC_flag"] = "hdim_2"
parameters_linking["min_h2"] = 0
parameters_linking["max_h2"] = len(longitude) - 1

In [13]:
parameters_linking["v_max"] = 9.0

Track1 = tobac.linking_trackpy(Features1, olr1_standardised, dt=dt, dxy=dxy, **parameters_linking)

Frame 719: 6 trajectories present.


In [14]:
parameters_linking["v_max"] = 27.0

Track2 = tobac.linking_trackpy(Features2, olr2_standardised, dt=dt, dxy=dxy, **parameters_linking)

Frame 719: 12 trajectories present.


In [15]:
parameters_linking["v_max"] = 50.0

Track3 = tobac.linking_trackpy(Features3, olr3_standardised, dt=dt, dxy=dxy, **parameters_linking)

Frame 719: 83 trajectories present.


In [16]:
parameters_segmentation = {}
parameters_segmentation["target"] = "minimum"
parameters_segmentation["method"] = "watershed"
parameters_segmentation["threshold"] = -2.0
parameters_segmentation["PBC_flag"] = "hdim_2"

In [17]:
Mask1, Track1 = tobac.segmentation_2D(Track1, olr1_standardised, dxy, **parameters_segmentation)

INFO:root:Start watershedding


In [18]:
Mask2, Track2 = tobac.segmentation_2D(Track2, olr2_standardised, dxy, **parameters_segmentation)

INFO:root:Start watershedding


In [19]:
Mask3, Track3 = tobac.segmentation_2D(Track3, olr3_standardised, dxy, **parameters_segmentation)

INFO:root:Start watershedding


In [20]:
time_plot_start = "2021-01-01 00:00:00"
time_plot_end = "2021-01-03 21:00:00"

time_plot = time.sel(time=slice(time_plot_start, time_plot_end))

In [22]:
figx = 12.0
figy = 14.0

fig = plt.figure(figsize=(figx, figy))
plt.close()

def update(frame):
    fig.clf()
    time_frame = np.datetime_as_string(time_plot.values[frame], unit="s")
    
    olr1_plot = olr1.sel(time=time_frame)
    olr2_plot = olr2.sel(time=time_frame)
    olr3_plot = olr3.sel(time=time_frame)
    
    Features1_plot = Features1.loc[pd.to_datetime(Features1["timestr"]) == pd.to_datetime(time_frame)]
    Features2_plot = Features2.loc[pd.to_datetime(Features2["timestr"]) == pd.to_datetime(time_frame)]
    Features3_plot = Features3.loc[pd.to_datetime(Features3["timestr"]) == pd.to_datetime(time_frame)]
    
    Mask1_plot = Mask1.sel(time=time_frame)
    Mask2_plot = Mask2.sel(time=time_frame)
    Mask3_plot = Mask3.sel(time=time_frame)
    
    Mask1_plot, lon1_plot = cutil.add_cyclic(Mask1_plot, x=Mask1_plot["longitude"])
    Mask2_plot, lon2_plot = cutil.add_cyclic(Mask2_plot, x=Mask2_plot["longitude"])
    Mask3_plot, lon3_plot = cutil.add_cyclic(Mask3_plot, x=Mask3_plot["longitude"])
    
    ax1 = fig.add_axes([1.0 / figx, 10.0 / figy, 10.0 / figx, 3.0 / figy], projection=ccrs.PlateCarree(central_longitude=210.0), aspect="auto")
    plot1 = ax1.pcolormesh(olr1_plot["longitude"], olr1_plot["latitude"], olr1_plot, transform=ccrs.PlateCarree(), shading="nearest", cmap="RdBu_r", vmin=-60.0, vmax=60.0, edgecolors="face")
    plot1_boundary = ax1.contour(lon1_plot, olr1_plot["latitude"], Mask1_plot, transform=ccrs.PlateCarree(), levels=[0.5], colors="black")
    plot1_marker = ax1.scatter(x=Features1_plot["longitude"], y=Features1_plot["latitude"], s=15, c="yellow", marker="x", transform=ccrs.PlateCarree())
    
    ax1.coastlines()
    ax1.set_title("(a) Moisture Mode", loc="left", fontsize=18.0)
    ax1.set_title(time_frame, loc="right", fontsize=18.0)
    ax1.set_xticks(np.arange(0.0, 360.0, 60.0), crs=ccrs.PlateCarree())
    ax1.set_yticks(np.arange(-30.0, 45.0, 15.0), crs=ccrs.PlateCarree())
    ax1.tick_params(labelsize=18.0)
    ax1.xaxis.set_major_formatter(LongitudeFormatter(zero_direction_label=True))
    ax1.yaxis.set_major_formatter(LatitudeFormatter())
    
    ax2 = fig.add_axes([1.0 / figx, 6.0 / figy, 10.0 / figx, 3.0 / figy], projection=ccrs.PlateCarree(central_longitude=210.0), aspect="auto")
    plot2 = ax2.pcolormesh(olr2_plot["longitude"], olr2_plot["latitude"], olr2_plot, transform=ccrs.PlateCarree(), shading="nearest", cmap="RdBu_r", vmin=-60.0, vmax=60.0, edgecolors="face")
    plot2_boundary = ax2.contour(lon2_plot, olr2_plot["latitude"], Mask2_plot, transform=ccrs.PlateCarree(), levels=[0.5], colors="black")
    plot2_marker = ax2.scatter(x=Features2_plot["longitude"], y=Features2_plot["latitude"], s=15, c="yellow", marker="x", transform=ccrs.PlateCarree())
    
    ax2.coastlines()
    ax2.set_title("(b) Mixed System", loc="left", fontsize=18.0)
    ax2.set_title(time_frame, loc="right", fontsize=18.0)
    ax2.set_xticks(np.arange(0.0, 360.0, 60.0), crs=ccrs.PlateCarree())
    ax2.set_yticks(np.arange(-30.0, 45.0, 15.0), crs=ccrs.PlateCarree())
    ax2.tick_params(labelsize=18.0)
    ax2.xaxis.set_major_formatter(LongitudeFormatter(zero_direction_label=True))
    ax2.yaxis.set_major_formatter(LatitudeFormatter())
    
    ax3 = fig.add_axes([1.0 / figx, 2.0 / figy, 10.0 / figx, 3.0 / figy], projection=ccrs.PlateCarree(central_longitude=210.0), aspect="auto")
    plot3 = ax3.pcolormesh(olr3_plot["longitude"], olr3_plot["latitude"], olr3_plot, transform=ccrs.PlateCarree(), shading="nearest", cmap="RdBu_r", vmin=-60.0, vmax=60.0, edgecolors="face")
    plot3_boundary = ax3.contour(lon3_plot, olr3_plot["latitude"], Mask3_plot, transform=ccrs.PlateCarree(), levels=[0.5], colors="black")
    plot3_marker = ax3.scatter(x=Features3_plot["longitude"], y=Features3_plot["latitude"], s=15, c="yellow", marker="x", transform=ccrs.PlateCarree())
    
    ax3.coastlines()
    ax3.set_title("(c) IG Wave", loc="left", fontsize=18.0)
    ax3.set_title(time_frame, loc="right", fontsize=18.0)
    ax3.set_xticks(np.arange(0.0, 360.0, 60.0), crs=ccrs.PlateCarree())
    ax3.set_yticks(np.arange(-30.0, 45.0, 15.0), crs=ccrs.PlateCarree())
    ax3.tick_params(labelsize=18.0)
    ax3.xaxis.set_major_formatter(LongitudeFormatter(zero_direction_label=True))
    ax3.yaxis.set_major_formatter(LatitudeFormatter())
    
    cb_ax = fig.add_axes([0.25, 0.75 / figy, 0.5, 0.5 * figx / 20 / figy])
    
    cb = fig.colorbar(plot1, cax=cb_ax, orientation="horizontal", extend="both")
    
    cb_ax.set_xlabel("OLR Anomaly [W m**-2]", fontsize=18.0)
    cb_ax.tick_params(labelsize=18.0)

animation = ani.FuncAnimation(fig=fig, func=update, frames=len(time_plot), interval=750)

#HTML(animation.to_html5_video())
animation.save(filename="/scratch/k10/mr4682/hk25-AusNode-ConvTrop/plots/tobac_examples.mp4")

INFO:matplotlib.animation:Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
INFO:matplotlib.animation:MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 1200x1400 -pix_fmt rgba -framerate 1.3333333333333333 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -y /scratch/k10/mr4682/hk25-AusNode-ConvTrop/plots/tobac_examples.mp4


In [ ]:
Track = Track.loc[(pd.to_datetime(Track["timestr"].values) >= pd.to_datetime(time_start)) & (pd.to_datetime(Track["timestr"].values) <= pd.to_datetime(time_end))]